# LLM-powered AI Agents

Table of contents
1. Understanding LLMs
2. Tools
3. Chat-based AI Agents
4. Service-based AI agents
5. Multi-Agents

In [1]:
from language_models.proxy_client import BTPProxyClient
from language_models.settings import settings

proxy_client = BTPProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

## 1. Understanding LLMs

In [2]:
from language_models.models.llm import OpenAILanguageModel, ChatMessage, ChatMessageRole

In [3]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-35-turbo",
    max_tokens=256,
    temperature=0.0,
)

In [4]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!"""

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

Sentiment: Positive


In [5]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!

Respond with positive or negative."""

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

positive


In [6]:
system_prompt = "Take the following movie review and determine the sentiment of the review. Respond with 1 (positive) or 0 (negative)."

prompt = "Wow! This movie was incredible. The acting was superb, and the plot kept me on the edge of my seat. I highly recommend it!"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

1


In [7]:
system_prompt = "Take the following movie review determine the sentiment of the review. Respond with 1 (positive) or 0 (negative)."

prompt = "Will it rain in Seattle today?"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

I'm sorry, I am an AI language model and I do not have access to real-time weather information. I recommend checking a reliable weather website or using a weather app to get the most accurate and up-to-date forecast for Seattle.


In [8]:
system_prompt = """Take the following movie review and determine the sentiment of the review. 

Respond with 1 (positive) or 0 (negative).

If you don't receive a movie review, respond with -1."""

prompt = "Will it rain in Seattle today?"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

-1


## 2. Tools

In [9]:
import json
from language_models.tools.tool import Tool
from pydantic import BaseModel, Field
from typing import Any

In [10]:
prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12" # answer: $1,289.98

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

Total Raw Cost = $1,290.98


In [11]:
def calculator(expression: str) -> Any:
    return eval(expression)

class Calculator(BaseModel):
    expression: str = Field(description="A math expression.")

calculator_tool = Tool(
    func=calculator,
    name="Calculator",
    description="Use this tool when you want to do calculations.",
    args_schema=Calculator
)
print(calculator_tool)

tool name: Calculator, tool description: Use this tool when you want to do calculations., tool input: {{'expression': {{'description': 'A math expression.', 'title': 'Expression', 'type': 'string'}}}}


In [12]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible.

You have access to the following tools: {calculator_tool}

Use a JSON blob to specify a thought, a tool by providing an tool key (tool name) and a tool_input key (tool input).

Valid "tool" values: {calculator_tool.name}

Always use the following JSON format:
{{
  "thought": "You should always think about what to do consider previous and subsequent steps",
  "tool": "The tool to use",
  "tool_input": "Valid key value pairs",
}}"""

prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
response = json.loads(response, strict=False)
print(json.dumps(response, indent=4))

{
    "thought": "To calculate the total raw cost, you need to add up all the individual costs.",
    "tool": "Calculator",
    "tool_input": {
        "expression": "549.72 + 6.98 + 41.00 + 35.00 + 552.00 + 76.16 + 29.12"
    }
}


In [13]:
print(calculator(**response["tool_input"]))

1289.98


In [14]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible.

You have access to the following tools: {calculator_tool}

Use a JSON blob to specify a thought, a tool by providing an tool key (tool name) and a tool_input key (tool input).

Valid "tool" values: {calculator_tool.name}

Always use the following JSON format:
{{
  "thought": "You should always think about what to do consider previous and subsequent steps",
  "tool": "The tool to use",
  "tool_input": "Valid key value pairs",
}}

Observation: tool result
... (this Thought/Tool/Observation can repeat N times)

When you know the answer, use the following JSON format:
{{
  "thought": "I now know what to respond",
  "tool": "Final Answer",
  "tool_input": "The final answer to the question",
}}"""

prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
    ChatMessage(role=ChatMessageRole.ASSISTANT, content=json.dumps(response)),
    ChatMessage(role=ChatMessageRole.ASSISTANT, content=f"Response of Calculator tool: {calculator(**response['tool_input'])}"),
])
response = json.loads(response, strict=False)
print(json.dumps(response, indent=4))

{
    "thought": "I now know the total raw cost.",
    "tool": "Final Answer",
    "tool_input": "The total raw cost is $1289.98."
}


## 3. Chat-based AI Agents

In [15]:
from language_models.agents.react import ReActAgent
from language_models.tools.earthquake import earthquake_tools
from language_models.tools.current_date import current_date_tool

### Earthquake

In [16]:
system_prompt = """You are an United States Geological Survey expert who can answer questions regarding earthquakes and can run forecasts.

Use the current date tool to access the local date and time before using other tools.

Take the following question and answer it as accurately as possible.
"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4-32k',
    max_tokens=2048,
    float=0.0,
)

class Output(BaseModel):
    content: str = Field(description="The final answer.")

earthquake_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="{question}",
    task_prompt_variables=["question"],
    tools=earthquake_tools + [current_date_tool],
    output_format=Output,
    iterations=10,
)

In [17]:
response = earthquake_agent.invoke({"question": "How many earthquakes have occurred for the past week with a magnitude of 5 or greater?"})

07/05/24 21:27:33 INFO Prompt:
How many earthquakes have occurred for the past week with a magnitude of 5 or greater?
07/05/24 21:27:37 INFO Raw response:
{
  "thought": "First, I need to get the current date to calculate the start date for the past week.",
  "tool": "Current Date",
  "tool_input": {}
}
07/05/24 21:27:37 INFO Thought:
First, I need to get the current date to calculate the start date for the past week.
07/05/24 21:27:37 INFO Tool:
Current Date
07/05/24 21:27:37 INFO Tool input:
{}
07/05/24 21:27:37 INFO Tool response:
2024-05-07 21:27:37.528359
07/05/24 21:27:44 INFO Raw response:
{
  "thought": "Now that I have the current date, I can calculate the start date for the past week. Then, I will use the 'Count' tool to find out how many earthquakes with a magnitude of 5 or greater have occurred during this period.",
  "tool": "Count",
  "tool_input": {
    "start_time": "2024-04-30T21:27:37.528359",
    "min_magnitude": 5
  }
}
07/05/24 21:27:44 INFO Thought:
Now that I hav

In [18]:
print(response.final_answer["content"])

There have been 34 earthquakes with a magnitude of 5 or greater in the past week.


In [19]:
response = earthquake_agent.invoke({"question": "Query 10 earthquakes that occurred yesterday and have a magnitude > 3."})

07/05/24 21:27:50 INFO Prompt:
Query 10 earthquakes that occurred yesterday and have a magnitude > 3.
07/05/24 21:27:55 INFO Raw response:
Tool response:
2024-05-08 21:27:37.528359
07/05/24 21:27:58 INFO Raw response:
Tool response:
2024-05-07 21:27:37.528359
07/05/24 21:28:04 INFO Raw response:
{
  "thought": "I need to get the current date first to calculate the date for 'yesterday'.",
  "tool": "Current Date",
  "tool_input": {}
}
07/05/24 21:28:04 INFO Thought:
I need to get the current date first to calculate the date for 'yesterday'.
07/05/24 21:28:04 INFO Tool:
Current Date
07/05/24 21:28:04 INFO Tool input:
{}
07/05/24 21:28:04 INFO Tool response:
2024-05-07 21:28:04.177000
07/05/24 21:28:17 INFO Raw response:
{
  "thought": "I need to query the earthquakes that occurred yesterday with a magnitude greater than 3. I will use the 'Query' tool with the 'start_time' set to the date of yesterday, 'end_time' set to the current date, 'min_magnitude' set to 3, and 'limit' set to 10.",


In [20]:
print(response.final_answer["content"])

Here are 10 earthquakes that occurred yesterday with a magnitude greater than 3:

1. [M 4.5 - 42 km NNW of Poso, Indonesia](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mwur)
2. [M 3.0 - 21 km SW of Jal, New Mexico](https://earthquake.usgs.gov/earthquakes/eventpage/tx2024ixyc)
3. [M 4.8 - 206 km NE of Lospalos, Timor Leste](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mwt7)
4. [M 4.9 - 131 km NW of Sola, Vanuatu](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mwst)
5. [M 4.8 - 21 km ESE of San Gabriel, Ecuador](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mwq2)
6. [M 5.0 - 35 km ESE of San Gabriel, Ecuador](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mwpy)
7. [M 4.8 - 22 km SE of Chimoré, Bolivia](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mwpv)
8. [M 4.8 - 12 km NE of Montelíbano, Colombia](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mwpt)
9. [M 3.0 - 7 km NNE of Tierras Nuevas Poniente, Puerto Rico](https:

In [21]:
response = earthquake_agent.invoke({"question": "Can MegaQuakes really happen? Like a magnitude 10 or larger?"})

07/05/24 21:29:00 INFO Prompt:
Can MegaQuakes really happen? Like a magnitude 10 or larger?
07/05/24 21:29:51 INFO Raw response:
{
  "thought": "The user is asking about the possibility of a magnitude 10 or larger earthquake, also known as a 'MegaQuake'. The largest recorded earthquake in history was a magnitude 9.5 in Chile in 1960. A magnitude 10 earthquake would release 31.6 times more energy than the 9.5 earthquake. Such an event would require a fault of the length of nearly the entire earth's circumference. It's theoretically possible, but the earth's crust is not believed to have the strength to generate and sustain such a large rupture in a single event. Therefore, it's highly unlikely.",
  "tool": "Final Answer",
  "tool_input": {"content": "The largest recorded earthquake in history was a magnitude 9.5 in Chile in 1960. A magnitude 10 earthquake, also known as a 'MegaQuake', would release 31.6 times more energy than the 9.5 earthquake. Such an event would require a fault of th

In [22]:
print(response.final_answer["content"])

The largest recorded earthquake in history was a magnitude 9.5 in Chile in 1960. A magnitude 10 earthquake, also known as a 'MegaQuake', would release 31.6 times more energy than the 9.5 earthquake. Such an event would require a fault of the length of nearly the entire earth's circumference. It's theoretically possible, but the earth's crust is not believed to have the strength to generate and sustain such a large rupture in a single event. Therefore, while a magnitude 10 or larger earthquake is theoretically possible, it's highly unlikely to occur.


## 4. Service-based AI Agents

In [23]:
import pandas as pd
from sklearn.metrics import accuracy_score

### Sentiment Analysis

In [24]:
df_tweets = pd.read_csv("./data/tweets.csv.gz", compression="gzip", encoding="latin-1", names=["sentiment", "id", "date", "query", "user", "tweet"])
df_tweets = df_tweets.dropna()
df_tweets = df_tweets.where(df_tweets.sentiment != 2)
df_tweets["sentiment"] = df_tweets["sentiment"].map({4: 1, 0: 0})
df_tweets_sampled = df_tweets.sample(n=10)
df_tweets_sampled.head()

,sentiment,id,date,query,user,tweet
315974,0,2002308188,Tue Jun 02 03:49:43 PDT 2009,NO_QUERY,Kissten12,is going to cheer . . . I mean dance. . Its to...
787912,0,2325014349,Thu Jun 25 04:34:28 PDT 2009,NO_QUERY,Animaleante,@fabianelima no youtube for me at work
1283209,1,2001916838,Tue Jun 02 02:31:14 PDT 2009,NO_QUERY,lindsaylumbera,hi guys! im back twitting. ;)
1155280,1,1978997356,Sun May 31 01:13:41 PDT 2009,NO_QUERY,RaspberryDoodle,"Sunday morning, the sun is shining and I am of..."
871964,1,1678849253,Sat May 02 07:53:17 PDT 2009,NO_QUERY,Ford95,Thought cancer bats were fuknig awesome night ...


In [25]:
system_prompt = """Take the following tweet and determine the sentiment of the review. 

Respond with 1 (positive) or 0 (negative).

If you don't receive a tweet, respond with -1.
"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=128,
    float=0.0,
)

class Output(BaseModel):
    sentiment: int = Field(description="The sentiment of the tweet.")

sentiment_analysis_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Tweet:\n{tweet}",
    task_prompt_variables=["tweet"],
    tools=None,
    output_format=Output,
    iterations=2,
)

In [26]:
def classify_sentiment(tweet: str) -> int:
    response = sentiment_analysis_agent.invoke({'tweet': tweet})
    return response.final_answer['sentiment'] or 0

In [27]:
df_tweets_sampled["prediction"] = [classify_sentiment(tweet) for tweet in df_tweets_sampled.tweet]

07/05/24 21:29:55 INFO Prompt:
Tweet:
is going to cheer . . . I mean dance. . Its too early 
07/05/24 21:30:03 INFO Raw response:
{
  "thought": "The tweet seems to express a positive sentiment as the user is going to cheer or dance, although they mention it's too early, it doesn't seem to express negativity.",
  "tool": "Final Answer",
  "tool_input": {"sentiment": 1}
}
07/05/24 21:30:03 INFO Thought:
The tweet seems to express a positive sentiment as the user is going to cheer or dance, although they mention it's too early, it doesn't seem to express negativity.
07/05/24 21:30:03 INFO Final answer:
{'sentiment': 1}
07/05/24 21:30:03 INFO Prompt:
Tweet:
@fabianelima  no youtube for me at work
07/05/24 21:30:08 INFO Raw response:
{
  "thought": "The tweet seems to express a negative sentiment as the user is unable to access YouTube at work.",
  "tool": "Final Answer",
  "tool_input": {"sentiment": 0}
}
07/05/24 21:30:08 INFO Thought:
The tweet seems to express a negative sentiment as t

In [28]:
print(f"Accuracy: {accuracy_score(df_tweets_sampled.sentiment, df_tweets_sampled.prediction)}")

Accuracy: 0.6


### Structuring Unstructured Data

In [29]:
import random
from pathlib import Path
from datetime import datetime

In [30]:
path = Path("./data/jobs")
filenames = [file.name for file in path.iterdir() if file.is_file()]
filenames = random.sample(filenames, 5)

jobs = []
for filename in filenames:
    file_path = path / filename
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()
        jobs.append(content)

In [31]:
system_prompt = """Take the following job and extract data about the job. 

Respond with the following extracted data:
- job_title: The job title.
- job_class_no: The job class code.
- job_duties: The duties of the job.
- open_date: When the position was opened. Format: DD-MM-YYYY.
- salary: The salary ranges. Format: 'min salary to max salary'.
- deadline: The application deadline. Format: DD-MM-YYYY.
- application_form: The form of the application (e.g. online, fax, email).
- where_to_apply: The url to apply at or location to send the fax or email address.
"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1024,
    float=0.0,
)

class JobData(BaseModel):
    job_title: str = Field(description="The job title.")
    job_class_no: str = Field(description="The job class code.")
    job_duties: str = Field(description="The duties of the job.")
    open_date: str = Field(description="When the position was opened. Format: DD-MM-YYYY.", max_length=10)
    salary: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")
    deadline: str = Field(description="The application deadline. Format: DD-MM-YYYY", max_length=10)
    application_form: str = Field(description="The form of the application (e.g. online, fax, email).")
    where_to_apply: str = Field(description="The url to apply at or location to send the fax or email address.")

job_data_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Job description:\n{job}",
    task_prompt_variables=["job"],
    tools=None,
    output_format=JobData,
    iterations=10,
)

In [32]:
def extract_jobs(jobs: list[str]) -> pd.DataFrame:
    data = []
    for job in jobs:
        response = job_data_agent.invoke({"job": job})
        data.append(response.final_answer)
    return pd.DataFrame(data)

In [33]:
df_jobs = extract_jobs(jobs)

07/05/24 21:30:55 INFO Prompt:
Job description:
PRINCIPAL SECURITY OFFICER

Class Code:        3200
Open Date: 11-02-18
(Exam Open to Current City Employees)

ANNUAL SALARY

$49,882 to $ 72,975
The salary in the Department of Water and Power is $69,780 to $86,693

NOTES:

1. For information regarding reciprocity between the City of Los Angeles departments and LADWP, go to: http://per.lacity.org/Reciprocity_CityDepts_and_DWP.pdf.
2. The current salary range is subject to change. You may confirm the starting salary with the hiring department before accepting a job offer.
3. In some positions, the salary is higher for night work.

DUTIES

A Principal Security Officer assists in directing, or personally directs, a large group of Security Officers and Senior Security Officers engaged in patrolling and safeguarding buildings and their occupants, grounds and equipment, in crowd and traffic control and protocol duties; supervises the investigation of accidents, thefts and disturbances; applies

In [34]:
df_jobs.head()

,job_title,job_class_no,job_duties,open_date,salary,deadline,application_form,where_to_apply
0,PRINCIPAL SECURITY OFFICER,3200,A Principal Security Officer assists in direct...,11-02-2018,"[$49,882 to $72,975, $69,780 to $86,693]",15-11-2018,online,http://www.governmentjobs.com/careers/lacity/p...
1,Blacksmith,3733,"A blacksmith forges, shapes, forms, bends, cut...",25-08-2017,"[$83,352 (flat-rated), $85,023 (flat-rated), $...",14-09-2017,online,https://www.governmentjobs.com/careers/lacity
2,Water Utility Worker,3912,"A Water Utility Worker installs, maintains, op...",12-08-2017,"[$67,275 to $83,582, $72,537 to $90,138, $88,1...",Filing may be closed without prior notice afte...,Online,https://www.governmentjobs.com/careers/lacity
3,SENIOR ELECTRIC SERVICE REPRESENTATIVE,7521,A Senior Electric Service Representative plans...,29-09-2017,"[$108,805 to $114,881, $115,194 to $ 121,626, ...",12-10-2017,online,https://www.governmentjobs.com/careers/lacity/...
4,FIREBOAT MATE,5125,Operates a small fireboat in Los Angeles Harbo...,23-10-2015,"[$88,426 to $104,128]",12-11-2015,online,http://agency.governmentjobs.com/lacity/defaul...


### Comparing Unstructured Data

### Contract Drafting

## 5. Multi-Agents

In [35]:
from io import StringIO
from language_models.agents.chain import AgentChain

### Machine Learning Code Generation

In [36]:
system_prompt = """You are a Data Science agent, which helps the user solve machine learning problems.

Respond with 1 of the following machine learning problems:
- Classification
- Regression
- Clustering
- Time series forecasting"""

task_prompt = """Choose the machine learning problem best suited for the following problem and dataset.

Problem description: 
{problem_description}

Dataset:
Number of rows: {dataset_size}
Schema: 
{dataset_schema}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=128,
    float=0.0,
)

class ModelingProblem(BaseModel):
    modeling_problem: str = Field(description="The machine learning problem.")

problem_finder_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["problem_description", "dataset_size", "dataset_schema"],
    tools=None,
    output_format=ModelingProblem,
    iterations=5,
)

In [37]:
system_prompt = """You are a Data Science agent, which helps the user solve machine learning problems.

You can solve machine learning problems for:
- Classification
- Regression
- Clustering
- Time series forecasting

You have access to the following Python libraries:
- pandas
- numpy
- scikit-learn"""

task_prompt = """Given the following machine learning problem, respond with Python code.

Modeling problem: {modeling_problem}

Dataset:
Number of rows: {dataset_size}
Schema: 
{dataset_schema}
First 10 rows of dataset:
{dataset_snippet}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=2048,
    float=0.0,
)

class AutoMLCode(BaseModel):
    code: str = Field(description="The Python machine learning code.")

auto_ml_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["modeling_problem", "dataset_size", "dataset_schema", "dataset_snippet"],
    tools=None,
    output_format=AutoMLCode,
    iterations=10,
)

In [38]:
chain = AgentChain(
    chain=[problem_finder_agent, auto_ml_agent],
    chain_variables=["problem_description", "dataset_size", "dataset_schema", "dataset_snippet"]
)

In [39]:
info_str = StringIO()
df_tweets.info(buf=info_str)
dataset_schema = info_str.getvalue()

In [40]:
response = chain.invoke({
    "problem_description": "I want to classify the sentiment of tweets.", 
    "dataset_size": len(df_tweets), 
    "dataset_schema": dataset_schema, 
    "dataset_snippet": str(df_tweets.head(10).to_markdown())
})

07/05/24 21:33:40 INFO Prompt:
Choose the machine learning problem best suited for the following problem and dataset.

Problem description: 
I want to classify the sentiment of tweets.

Dataset:
Number of rows: 1600000
Schema: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   id         1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   query      1600000 non-null  object
 4   user       1600000 non-null  object
 5   tweet      1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB

07/05/24 21:33:48 INFO Raw response:
{
  "thought": "Given the problem description and the dataset, it seems like the user wants to predict the sentiment of tweets, which is a categorical variable. This is a typical example of a classification problem.",
  "tool": "Final Answer",

In [41]:
print(response.final_answer["code"])

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Load the data
# df = pd.read_csv('data.csv')

# Preprocess the 'tweet' column
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['tweet'])

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.2, random_state=42)

# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


### Forecasting

In [42]:
from datetime import timedelta
from language_models.tools.forecasting import get_earthquakes_data, ml_model

In [43]:
class Forecast(BaseModel):
    start_time: str = Field(None, description='Limit to events on or after the specified start time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.')
    end_time: str = Field(None, description='Limit to events on or before the specified end time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.')

def forecast(start_time = None, end_time = None):
    if start_time is None:
        start_time = (datetime.now() - timedelta(days=30)).date()
    if end_time is None:
        end_time = (datetime.now().date())
    df = get_earthquakes_data('https://earthquake.usgs.gov/fdsnws/event/1/query?', start_time, end_time)
    df_pred = ml_model.predict(df)
    return {'predictions': df_pred.to_dict(orient='records')}

In [44]:
forecasting_tool = Tool(func=forecast, name='forecast', description='Test forecast model on real-time events.', args_schema=Forecast)

In [45]:
task_prompt = """Take the following question and determine the start and end time to respond with.

Question:
{question}
"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=256,
    float=0.0,
)

class DateRange(BaseModel):
    start_time: str = Field(description="Limit to events on or after the specified start time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.")
    end_time: str = Field(description="Limit to events on or before the specified end time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.")

time_wizard_agent = ReActAgent.create(
    llm=llm,
    system_prompt="",
    task_prompt=task_prompt,
    task_prompt_variables=["question"],
    tools=[current_date_tool],
    output_format=DateRange,
    iterations=5,
)

In [46]:
chain = AgentChain(
    chain=[time_wizard_agent, forecasting_tool], 
    chain_variables=["question"],
)

In [47]:
response = chain.invoke({"question": "Run a forecast using the past week as data."})

07/05/24 21:34:53 INFO Prompt:
Take the following question and determine the start and end time to respond with.

Question:
Run a forecast using the past week as data.

07/05/24 21:34:59 INFO Raw response:
{
  "thought": "To answer this question, I need to determine the current date first, then calculate the start and end time for the past week.",
  "tool": "Current Date",
  "tool_input": {}
}
07/05/24 21:34:59 INFO Thought:
To answer this question, I need to determine the current date first, then calculate the start and end time for the past week.
07/05/24 21:34:59 INFO Tool:
Current Date
07/05/24 21:34:59 INFO Tool input:
{}
07/05/24 21:34:59 INFO Tool response:
2024-05-07 21:34:59.025753
07/05/24 21:35:09 INFO Raw response:
{
  "thought": "The current date is 2024-05-07. To run a forecast using the past week as data, the start time would be 7 days before the current date and the end time would be the current date.",
  "tool": "Final Answer",
  "tool_input": {
    "start_time": "2024

In [48]:
print(response.final_answer['forecast'])

{'predictions': [{'time': Timestamp('2024-04-30 21:46:23.180000+0000', tz='UTC'), 'prediction': 1.652925, 'latitude': 58.2721666666667, 'longitude': -155.1625, 'mag': -0.13, 'id': 'av93005634', 'place': '88 km NNW of Karluk, Alaska', 'location': '88 km NNW of Karluk, Alaska'}, {'time': Timestamp('2024-04-30 21:51:21.360000+0000', tz='UTC'), 'prediction': 2.154171, 'latitude': 38.8376656, 'longitude': -122.8078308, 'mag': 0.83, 'id': 'nc74043541', 'place': '8 km WNW of Cobb, CA', 'location': '8 km WNW of Cobb, CA'}, {'time': Timestamp('2024-04-30 21:55:14.440000+0000', tz='UTC'), 'prediction': 2.933007, 'latitude': 19.378, 'longitude': -155.2045, 'mag': 1.42, 'id': 'hv74201587', 'place': '7 km SSE of Volcano, Hawaii', 'location': '7 km SSE of Volcano, Hawaii'}, {'time': Timestamp('2024-04-30 22:01:39.910000+0000', tz='UTC'), 'prediction': 1.948478, 'latitude': 61.0308333333333, 'longitude': -152.369833333333, 'mag': 0.18, 'id': 'av93005629', 'place': '66 km W of Tyonek, Alaska', 'locati